<a href="https://colab.research.google.com/github/MartynovychSerhii/Employment-Test-Tasks/blob/main/Dataforest/DataForest_test_Martynovych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install rapidfuzz

In [24]:
# loading packages and files
import pandas as pd
from google.colab import drive
from rapidfuzz import process, fuzz


url1 = 'https://raw.githubusercontent.com/MartynovychSerhii/Employment-Test-Tasks/refs/heads/main/Dataforest/company_dataset_1.csv'
url2 = 'https://raw.githubusercontent.com/MartynovychSerhii/Employment-Test-Tasks/refs/heads/main/Dataforest/company_dataset_2.csv'


try:
    df1 = pd.read_csv(url1, engine='python')
    df2 = pd.read_csv(url2, engine='python')
    print("Files loaded successfully from GitHub!")
except Exception as e:
    print(f"Error loading files: {e}")

Files loaded successfully from GitHub!


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   custnmbr    1050 non-null   object
 1   addrcode    1050 non-null   object
 2   custname    1050 non-null   object
 3   sStreet1    1047 non-null   object
 4   sStreet2    160 non-null    object
 5   sCity       1048 non-null   object
 6   sProvState  1046 non-null   object
 7   sCountry    165 non-null    object
 8   sPostalZip  1035 non-null   object
dtypes: object(9)
memory usage: 74.0+ KB


In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123 entries, 0 to 1122
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   custnmbr  1123 non-null   object
 1   addrcode  1107 non-null   object
 2   custname  1123 non-null   object
 3   address1  1123 non-null   object
 4   address2  1123 non-null   object
 5   address3  1123 non-null   object
 6   ccode     1123 non-null   object
 7   city      1123 non-null   object
 8   country   1123 non-null   object
 9   state     1123 non-null   object
 10  zip       1123 non-null   object
dtypes: object(11)
memory usage: 96.6+ KB


In [27]:
# cleaning data and matching addresses
def clean_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower().strip()

    for suffix in [' inc.', ' inc', ' inc.-', 'inc.*', ' ltd.', ' ltd', ' corp.', ' llc.', '.', ',', '*', 'lp']:
        name = name.replace(suffix, '')
    return name.strip()

df1['clean_name'] = df1['custname'].apply(clean_name)
df2['clean_name'] = df2['custname'].apply(clean_name)

ds1_unique = df1.groupby('custname').agg({
    'sStreet1': lambda x: list(x.dropna()),
    'sCity': 'first',
    'sProvState': 'first',
    'sPostalZip': 'first',
    'clean_name': 'first'
}).reset_index()

ds2_unique = df2.groupby('custname').agg({
    'address1': lambda x: [str(a).strip() for a in x.dropna()],
    'city': 'first',
    'zip': 'first',
    'clean_name': 'first'
}).reset_index()

choices = ds2_unique['clean_name']

def get_match_info(name):
    match = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= 85:
        return ds2_unique.iloc[match[2]]['custname'], ds2_unique.iloc[match[2]]['address1']
    return None, []

matches = ds1_unique['clean_name'].apply(get_match_info)
ds1_unique['matched_name_ds2'] = matches.apply(lambda x: x[0])
ds1_unique['locations_ds2'] = matches.apply(lambda x: x[1])

ds1_unique = ds1_unique.rename(columns={'sStreet1': 'locations_ds1'})

def find_overlap(row):
    loc1 = set([str(a).lower().strip() for a in row['locations_ds1']])
    loc2 = set([str(a).lower().strip() for a in row['locations_ds2']])
    overlap = list(loc1.intersection(loc2))
    return overlap if overlap else None

ds1_unique['overlapping_locations'] = ds1_unique.apply(find_overlap, axis=1)

final_df = ds1_unique[['custname', 'matched_name_ds2', 'locations_ds1', 'locations_ds2', 'overlapping_locations']]
final_df

,custname,matched_name_ds2,locations_ds1,locations_ds2,overlapping_locations
0,110 CVPW LP,None,[100 Commerce Valley Drive W.],[],None
1,110/140 Coxommepa Road Taguomex Inc.,None,[200-1737 Woodward Drive],[],None
2,1106832 Ontario Ltd.,None,"[277 Lakeshore Road E, Suite 200]",[],None
3,1111429 Ontario Inc.,1111429 Ontario Inc. ...,[100 - 2600 Finch Avenue West],[100 - 2600 Finch Avenue West],[100 - 2600 finch avenue west]
4,1132 International Monumaa Inc.,1132 INTERNATIONAL MONUMAA INC ...,[2275 Upper Middle Road East Suite 700],[C/O FENGATE PROPERTY MGMT LTD],None
...,...,...,...,...,...
1045,Épufuca 211 Xeutuat Inc.,Épufuca 211 Xeutuat Inc. ...,[630 Rue Saint-Paul Ouest],[630 Rue Saint-Paul Ouest],[630 rue saint-paul ouest]
1046,Épufuca 3510 St-Jecquaz,None,"[630, rue Saint-Paul Ouest]",[],None
1047,Épufuca 5110 Zhatbtooka Inc.,Épufuca 5110 Zhatbtooka Inc. ...,[630 Rue Saint-Paul Ouest],[630 Rue Saint-Paul Ouest],[630 rue saint-paul ouest]
1048,Épufuca 635 Tamé-Xévazqua Ouazt Inc.,Épufuca 635 Tamé-Xévazqua Ouazt Inc. ...,[630 rue Saint-Paul Ouest #600],[630 rue Saint-Paul Ouest #600],[630 rue saint-paul ouest #600]


In [28]:
# Making metrics
matched_count = final_df['matched_name_ds2'].notna().sum()
total_ds1 = len(final_df)
match_rate = (matched_count / total_ds1) * 100

unmatched_count = final_df['matched_name_ds2'].isna().sum()
unmatched_rate = (unmatched_count / total_ds1) * 100


one_to_many_count = final_df[final_df['locations_ds2'].apply(len) > 1].shape[0]
one_to_many_rate = (one_to_many_count / total_ds1) * 100

fully_verified = final_df['overlapping_locations'].notna().sum()
overlap_rate = (fully_verified / matched_count) * 100 if matched_count > 0 else 0

print(f"--- KEY METRICS ---")
print(f"Match Rate: {match_rate:.2f}% ({matched_count} from {total_ds1})")
print(f"Unmatched Records: {unmatched_rate:.2f}% ({unmatched_count} without a pair)")
print(f"One-to-Many (Multiple locations in DS2): {one_to_many_rate:.2f}%")
print(f"\n--- ADDITIONAL METRICS ---")
print(f"Location Overlap Rate: {overlap_rate:.2f}% (match both name and address)")

--- KEY METRICS ---
Match Rate: 49.33% (518 from 1050)
Unmatched Records: 50.67% (532 without a pair)
One-to-Many (Multiple locations in DS2): 6.38%

--- ADDITIONAL METRICS ---
Location Overlap Rate: 55.41% (match both name and address)


In [ ]:
# making csv
final_df.to_csv("final.csv")
from google.colab import files
files.download('final.csv')